In [ ]:
import numpy as np
import pandas
#import string
import urllib
from bs4 import BeautifulSoup as bs

#import dateutil
#from pprint import pprint
import os
import re
from datetime import datetime

os.chdir('/home/idies/workspace/Storage/raddick/raddick_folksonomy/data/')
#os.getcwd()
print('ok')

# Load data file

In [ ]:
# Read raw data
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df

# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read')

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

music_df[music_df['filename'].apply(lambda x: '.cue' in x)]

print('Step 0: remove that .cue file per Scott on 2018-02-08\n')
music_df = music_df[~music_df['filename'].apply(lambda x: '.cue' in x)]

music_df = music_df.assign(filetype = music_df['filename'].apply(lambda x: x.split('.')[-1].strip()))
music_df = music_df.assign(stem = music_df['filename'].apply(lambda x: x[0:x.rfind('.')]))


print('backing up...')
music_df_bk = music_df
#music_df.columns
print('ok')

# Make columns, Set some rows by hand (otherwise would screw up later)

In [ ]:
print('reading from backup...')

music_df = music_df_bk
music_df = music_df.assign(date_format = 'otros')
music_df = music_df.assign(date_string = np.nan)
music_df = music_df.assign(thedate = pandas.to_datetime(np.nan))

music_df = music_df.assign(theyear = np.nan)
music_df = music_df.assign(themonth = np.nan)
music_df = music_df.assign(theday = np.nan)

music_df.loc[music_df['stem'] == 'jgb1975-00-00t01'
             , ['date_format', 'date_string', 'theyear']] = ['byhand', '1975-00-00', 1975]
music_df.loc[music_df['stem'] == '01 - 4th of July (PA)-11 O\'clock tick tock'
             , ['date_format', 'date_string', 'themonth', 'theday']] = ['byhand', '4th of July', 7, 4]
music_df.loc[music_df['stem'] == '01 - 98 Elvin'
             , ['date_format', 'date_string', 'theyear', 'themonth']] = ['byhand', '01 - 98', 1998, 1]
music_df.loc[music_df['stem'] == '02-Peter Gabriel _ Paris 83 -CD2'
             , ['date_format', 'date_string', 'theyear']] = ['byhand', '83', 1983]
music_df.loc[music_df['stem'] == 'd1t01.The Ramones-Durango 95'
             , ['date_format', 'date_string', 'theyear']] = ['byhand', '95', 1995]
music_df.loc[music_df['stem'] == '21Ramones1981Aug20SCruzCivic', 
             ['date_format', 'date_string', 'thedate']] = ['byhand', '1981Aug20', pandas.to_datetime('1981-08-20')]
music_df.loc[music_df['stem'] == 'GORDON GATTON MFP 6 27 8101', 
             ['date_format', 'date_string', 'thedate']] = ['byhand', '6 27 81', pandas.to_datetime('1981-06-27')]
music_df.loc[music_df['stem'] == 'len price 3_2016_07_22_chatham_track_18',
            ['date_format', 'date_string', 'thedate']] = ['byhand', '2016_07_22', pandas.to_datetime('2016-07-22')]
music_df.loc[music_df['stem'] == 'BOB16070305',
            ['date_format', 'date_string', 'thedate']] = ['byhand', '160703', pandas.to_datetime('2016-07-03')]
music_df.loc[music_df['stem'] == 'SS19910700t01',
            ['date_format', 'date_string', 'theyear', 'themonth']] = ['byhand', '19910700', 1991, 7]
music_df.loc[music_df['stem'] == 'jaTRAB201618',
            ['date_format', 'date_string', 'thedate']] = ['byhand', '201618', pandas.to_datetime('2016-01-08')]
music_df.loc[music_df['stem'] == 'dt-000225_101',
            ['date_format', 'date_string', 'thedate']] = ['byhand', '000225', pandas.to_datetime('2000-02-25')]
music_df.loc[music_df['stem'] == '1997June22BeauSolielD1tr01',
            ['date_format', 'date_string', 'thedate']] = ['byhand', '1997June22', pandas.to_datetime('1997-06-22')]
music_df.loc[music_df['stem'] == '02 RRK_Village_Vanguard_75-0103 U T',
            ['date_format', 'date_string', 'thedate']] = ['byhand', '75-0103', pandas.to_datetime('1975-01-03')]
#

print('backing up...')
music_df_bk = music_df

#music_df.loc[1046]
music_df.groupby('date_format').size()



# Find filenames with proper yyyy-mm-dd date format

In [ ]:
print('reading from backup...')
music_df = music_df_bk

yyyymmddfinder = re.compile('(19|20)[\d]{2}\-[\d]{2}\-[\d]{2}')

music_df.loc[
    (music_df['date_format'] == 'otros')
    & (music_df['stem'].apply(lambda x: re.search(yyyymmddfinder,x) != None))
, 'date_format'] = 'yyyy-mm-dd'

music_df.loc[
    (music_df['date_format'] == 'yyyy-mm-dd')
, 'date_string'] = music_df['stem'][(music_df['date_format'] == 'yyyy-mm-dd')
                                   ].apply(lambda x: 
                                           x[re.search(yyyymmddfinder,x).start():re.search(yyyymmddfinder,x).end()]
                                          )
music_df.loc[(music_df['date_format'] == 'yyyy-mm-dd'), 'thedate'] = pandas.to_datetime(music_df['date_string'], errors='coerce')

print('backing up...')
music_df_bk = music_df

music_df.groupby('date_format').size()

#len(music_df['thedate'].dropna())

# Find filenames with date format <code>(19|20)[\d]{2}-[\d]{1,2}-[\d]{1,2}</code>

## First, compile this function, then move on to the next code cell.

In [ ]:
def parse_dates(dates_s):

    dates_df = pandas.DataFrame()
    dates_df = dates_df.assign(date_string = dates_s)

    dates_df = dates_df.assign(part1 = dates_df['date_string'].apply(lambda x: x.split('-')[0]))
    dates_df = dates_df.assign(part2 = dates_df['date_string'].apply(lambda x: x.split('-')[1]))
    dates_df = dates_df.assign(part3 = dates_df['date_string'].apply(lambda x: x.split('-')[2]))

    dates_df['part1'] = dates_df['part1'].apply(lambda x: int(x))
    dates_df['part2'] = dates_df['part2'].apply(lambda x: int(x))
    dates_df['part3'] = dates_df['part3'].apply(lambda x: int(x))   
    
    dates_df = dates_df.assign(year = np.nan)
    dates_df = dates_df.assign(month = np.nan)
    dates_df = dates_df.assign(day = np.nan)
    dates_df = dates_df.assign(date_format_2 = np.nan)
    
    dates_df.loc[
        (dates_df['year'].isnull())
        & (dates_df['part1'].apply(lambda x: x >= 1900))
         , 'year'] = dates_df['part1'].apply(lambda x: x)

    dates_df.loc[
        (dates_df['year'].isnull())
        & (dates_df['part1'].apply(lambda x: x < 1900))
        & (dates_df['part1'].apply(lambda x: x > 31))
         , 'year'] = dates_df['part1'].apply(lambda x: 1900 + x)
    dates_df.loc[
        (dates_df['month'].isnull())
        & (dates_df['part1'].apply(lambda x: x > 31))
         , 'month'] = dates_df['part2']
    dates_df.loc[
        (dates_df['day'].isnull())
        & (dates_df['part1'].apply(lambda x: x > 31))
         , 'day'] = dates_df['part3']
    dates_df.loc[
        (dates_df['date_format_2'].isnull())
        & ((dates_df['year'].notnull()) | (dates_df['month'].notnull()) | (dates_df['day'].notnull()) )
    , 'date_format_2'] = 'yy-mm-dd'

    
    dates_df.loc[
        (dates_df['year'].isnull())
        & (dates_df['part3'].apply(lambda x: x >= 1900))
         , 'year'] = dates_df['part1'].apply(lambda x: x)
    dates_df.loc[
        (dates_df['year'].isnull())
        & (dates_df['part3'].apply(lambda x: x > 31))
        & (dates_df['part3'].apply(lambda x: x < 1900))
        , 'year'] = dates_df['part3'].apply(lambda x: 1900 + x)
    dates_df.loc[
        (dates_df['month'].isnull())
        & (dates_df['part3'].apply(lambda x: x > 31))
        , 'month'] = dates_df['part1']
    dates_df.loc[
        (dates_df['day'].isnull())
        & (dates_df['part3'].apply(lambda x: x > 31))
        , 'day'] = dates_df['part2']
    dates_df.loc[
        (dates_df['date_format_2'].isnull())
        & ((dates_df['year'].notnull()) | (dates_df['month'].notnull()) | (dates_df['day'].notnull()) )
    , 'date_format_2'] = 'mm-dd-yy'


    dates_df.loc[   
        (dates_df['year'].isnull())
        & (dates_df['part2'] > 12)
        , 'year'] = dates_df['part3'].apply(lambda x: 2000 + x)
    dates_df.loc[   
        (dates_df['month'].isnull())
        & (dates_df['part2'] > 12)
        , 'month'] = dates_df['part1']
    dates_df.loc[   
        (dates_df['day'].isnull())
        & (dates_df['part2'] > 12)
        , 'day'] = dates_df['part2']
    dates_df.loc[
        (dates_df['date_format_2'].isnull())
        & ((dates_df['year'].notnull()) | (dates_df['month'].notnull()) | (dates_df['day'].notnull()) )
    , 'date_format_2'] = 'mm-dd-yy'


    dates_df.loc[
        (dates_df['year'].isnull())
        & (dates_df['part3'] > 12)
    , 'year'] = dates_df['part1'].apply(lambda x: 2000 + x)
    dates_df.loc[
        (dates_df['month'].isnull())
        & (dates_df['part3'] > 12)
    , 'month'] = dates_df['part2']
    dates_df.loc[
        (dates_df['day'].isnull())
        & (dates_df['part3'] > 12)
    , 'day'] = dates_df['part3']
    dates_df.loc[
        (dates_df['date_format_2'].isnull())
        & ((dates_df['year'].notnull()) | (dates_df['month'].notnull()) | (dates_df['day'].notnull()) )
    , 'date_format_2'] = 'yy-mm-dd'


    dates_df.loc[
        (dates_df['year'].isnull())
    , 'year'] = dates_df['part1'][(dates_df['year'].isnull())].apply(lambda x: 2000 + x)
    dates_df.loc[
        (dates_df['month'].isnull())
    , 'month'] = dates_df['part2'][(dates_df['month'].isnull())]
    dates_df.loc[
        (dates_df['day'].isnull())
    , 'day'] = dates_df['part3'][(dates_df['day'].isnull())]
    dates_df.loc[
        (dates_df['date_format_2'].isnull())
        & ((dates_df['year'].notnull()) | (dates_df['month'].notnull()) | (dates_df['day'].notnull()) )
    , 'date_format_2'] = 'yy-mm-dd'

    dates_df = dates_df.assign(thedate=pandas.to_datetime(dates_df[['year','month','day']], format='%Y-%m-%d', errors='coerce'))

#    return music_df
    return dates_df[['thedate','date_format_2']]

print('ok')

## Find filenames with date format <code>(19|20)[\d]{2}-[\d]{1,2}-[\d]{1,2}</code>

In [ ]:
print('reading from backup...')
music_df = music_df_bk
music_df = music_df.assign(date_format_2 = 'otros')

dashfinder = re.compile('[\d]{1,4}\-[\d]{1,2}\-[\d]{1,2}')

music_df.loc[
    (music_df['date_format'] == 'otros')    & (music_df['stem'].apply(lambda x: re.search(dashfinder,x) != None))
, 'date_format'] = 'dashfinder'

music_df.loc[
    (music_df['date_format'] == 'dashfinder')
, 'date_string'] = music_df['stem'][(music_df['date_format'] == 'dashfinder')
                                   ].apply(lambda x: 
                                           x[re.search(dashfinder,x).start():re.search(dashfinder,x).end()]
                                          )

print('parsing dates...')

music_df.loc[
    (music_df['date_format'] == 'dashfinder')
, ['thedate', 'date_format_2']] = parse_dates(music_df['date_string'][music_df['date_format'] == 'dashfinder'])

print('backing up...')
music_df_bk = music_df

print(music_df.groupby('date_format').size())

#music_df[['date_string','thedate']][music_df['date_format'] == 'dashfinder']

#music_df.loc[2298]

# Iterate through some other possible date formats

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

seps = re.compile('[,_\.\-\s]+')
ordinalios = re.compile('(st|nd|rd|th)*[\,]*[\s]+')

redict = {}
redict['longmonthname_day_year'] = re.compile('([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|J]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)[,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{2,4}')
redict['shortmonthname_day_year'] = re.compile('([j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[_,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{2,4}')
redict['day_longmonthname_year'] = re.compile('[\d]+(st|nd|rd|th)*[,\.\-\s]*([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|j]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)(?![a-z])[,\.\-\s]*[\d]{2,4}')
redict['day_shortmonthname_year'] = re.compile('[\d]+(st|nd|rd|th)*[,\.\-\s]*([j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[_,\.\-\s]*[\d]{2,4}')
redict['year_longmonthname_day'] = re.compile('([\d]+[,\.\-\s]*[j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|j]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)(?![a-z])[,\.\-\s]*[\d]+[st|nd|rd|th]*')
redict['year_shortmonthname_day'] = re.compile('([\d]+[,\.\-\s]*[j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[,\.\-\s]*[\d]+[st|nd|rd|th]*')
redict['eightstraight'] = re.compile('[\d]{8}')
redict['sixstraight'] = re.compile('[\d]{6}')
redict['other_numeric'] = re.compile('[\d]{1,4}[,_\.\-\s]+[\d]{1,4}[,_\.\-\s]+[\d]{1,4}')
redict['just_year_month'] = re.compile('(19|20)[\d]{2}[,_\.\-\s]+[\d]{1,2}')
redict['just_year'] = re.compile('(19|20)[\d]{2}')

keys_to_remove = []

for nameofthisre, thisre in redict.items():
    music_df.loc[
        (music_df['date_format'] == 'otros')
        & (music_df['stem'].apply(lambda x: re.search(thisre,x) != None))
    , 'date_format'] = nameofthisre
    music_df.loc[
        (music_df['date_format'] == nameofthisre)
    , 'date_string'] = music_df['stem'][
        (music_df['date_format'] == nameofthisre)
    ].apply(lambda x: x[re.search(thisre,x).start():re.search(thisre,x).end()])
    if (len(music_df[music_df['date_format'] == nameofthisre]) > 0):
        print('Regex {0:}: found {1:,.0f} matching filenames...'.format(nameofthisre, len(music_df[music_df['date_format'] == nameofthisre])))
    else:
        keys_to_remove.append(nameofthisre)
        
for x in keys_to_remove:
    del redict[x]

print('backing up...')
music_df_bk = music_df

music_df.groupby('date_format').size()

#music_df[music_df['date_string'] == '000225']



## Turn the date strings identified above into real dates

In [ ]:
print('reading from backup...')
music_df = music_df_bk

seps = re.compile('[,_\.\-\s]+')
print(redict.keys())
for k,v in redict.items():
    if (k in ['other_numeric']):
        music_df.loc[
            (music_df['date_format'] == k)
            , 'thedate'] = pandas.to_datetime(
            music_df['date_string'][music_df['date_format'] == k].apply(lambda x: 
                                                                        re.sub(seps,'-',x)
                                                                       ), errors='coerce')
    elif (k in ['eightstraight', 'sixstraight']):
        music_df.loc[music_df['date_format'] == k, 'thedate'] = pandas.to_datetime(music_df['date_string'], errors='coerce')
    elif (k in ['just_year_month', 'just_year']):
        pass
    else:
        music_df.loc[
            (music_df['date_format'] == k)
            , 'thedate'] = pandas.to_datetime(music_df['date_string'][
            (music_df['date_format'] == k)
        ].apply(lambda x: re.sub(ordinalios, ' ',x)), errors='coerce')
        
        
        
music_df.loc[music_df['date_format'] == 'twodigitdashfinder', 'date_format'] = music_df['date_format_2']
music_df = music_df.drop('date_format_2', axis=1)

music_df.loc[music_df['date_format'] == 'otros', 'date_format'] = 'nodate'

print('backing up...')
music_df_bk = music_df

#music_df#[['stem', 'date_format', 'date_format_2', 'date_string', 'thedate']][
#    music_df['date_format'] != 'otros'].sample(10)
#print(music_df[['date_string', 'thedate']][music_df['date_format'] == 'just_year_month'])
music_df.groupby('date_format').size()

### Check values with date format otros to make sure we didn't miss any

In [ ]:
# making sure we didn't miss any
#music_df['stem'][music_df['date_format'] == 'otros'].drop_duplicates().sort_values()#[900:] #939 total

# these were too long to tell at a glance
#for x in music_df['stem'].loc[[2317,2336,913,2971,2606,1906,2075,944,1875,1167,840]].tolist():
#    print(x)
print('ok')

### Get year, month, day from dates

In [ ]:
print('reading from backup...')
music_df = music_df_bk

music_df.loc[music_df['thedate'].notnull(), 'theyear'] = music_df['thedate'][music_df['thedate'].notnull()].apply(lambda x: x.year)
music_df.loc[music_df['thedate'].notnull(), 'themonth'] = music_df['thedate'][music_df['thedate'].notnull()].apply(lambda x: x.month)
music_df.loc[music_df['thedate'].notnull(), 'theday'] = music_df['thedate'][music_df['thedate'].notnull()].apply(lambda x: x.day)
             
music_df.loc[(music_df['date_format'] == 'just_year_month'), 'theyear'] = pandas.to_numeric(music_df['date_string'][(music_df['date_format'] == 'just_year_month')].apply(lambda x: re.sub(seps, '-', x).split('-')[0]))
music_df.loc[(music_df['date_format'] == 'just_year_month'), 'themonth'] = pandas.to_numeric(music_df['date_string'][(music_df['date_format'] == 'just_year_month')].apply(lambda x: re.sub(seps, '-', x).split('-')[1]))

music_df.loc[(music_df['date_format'] == 'just_year'), 'theyear'] = pandas.to_numeric(music_df['date_string'][(music_df['date_format'] == 'just_year')].apply(lambda x: re.sub(seps, '-', x).split('-')[0]))

print('backing up...')
music_df_bk = music_df

music_df[music_df['date_format'] == 'just_year'].sample(3)

# Parse into pre-date and post-date to get bands

In [ ]:
def get_pre_date_string(row):
    return row['stem'][0:re.search(row['date_string'], row['stem']).start()]

def get_post_date_string(row):
    return row['stem'][re.search(row['date_string'], row['stem']).end():len(row['stem'])]

endmarks = re.compile('[\.\-\_\ ]+$')
starts_with_number = re.compile('^[\d]+[\.\-\_\ \(\)\s]*')
thingy = re.compile('^d[\d]+t[\d]+\.')


bandseps = re.compile('[\s]*[,_\-\.]+[\s]*')


print('retrieving from backup...')
music_df = music_df_bk

music_df = music_df.assign(predate = music_df[music_df['date_format'] != 'nodate'].apply(lambda row: get_pre_date_string(row), axis=1))
music_df = music_df.assign(postdate = music_df[music_df['date_format'] != 'nodate'].apply(lambda row: get_post_date_string(row), axis=1))

music_df = music_df.assign(has_pre_date_endmarks = np.nan)

music_df.loc[
    (music_df['predate'].notnull())
    & (music_df['predate'][music_df['predate'].notnull()].apply(lambda x: 
                                                                re.search(endmarks, x) == None))
, 'has_pre_date_endmarks'] = False

music_df.loc[
    (music_df['predate'].notnull())
    & (music_df['predate'][music_df['predate'].notnull()].apply(lambda x: 
                                                                re.search(endmarks, x) != None))
, 'has_pre_date_endmarks'] = True

music_df = music_df.assign(band_abbrev = music_df['predate'][
    (music_df['predate'].notnull())
].apply(lambda x: re.sub(endmarks, '', x.lower())))


print('backing up...')
music_df_bk = music_df

print('ok')

### Fix a few bands whose abbreviations are hidden until further processing

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

loreley = re.compile('loreley')
lollapaloozafinder = re.compile('[\s_]*lollapalooza[\s_]*')



# Fix any supposed bands that start with a track number
music_df.loc[
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].dropna().apply(lambda x: re.search(starts_with_number,x) != None))
    , 'band_abbrev'] = music_df['band_abbrev'][
    (music_df['band_abbrev'].notnull()) 
    & (music_df['band_abbrev'].dropna().apply(lambda x: re.search(starts_with_number,x) != None))
].apply(lambda x: x[re.search(starts_with_number, x).end():len(x)])

# Fix any supposed bands that start with a d/t system (e.g. d1t01)
music_df.loc[
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(thingy, str(x)) != None))
, ['band_abbrev', 'has_pre_date_endmarks']] = [music_df['band_abbrev'][
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(thingy, str(x)) != None))
].apply(lambda x: x[re.search(thingy, x).end():len(x)]), True]

# Fix "loreley" filenames
music_df.loc[
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(loreley, str(x)) != None))
, ['band_abbrev', 'has_pre_date_endmarks']] = [music_df['band_abbrev'][
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(loreley, str(x)) != None))
].apply(lambda x: x[0:re.search(loreley, x).start()]), True]


doubledashfinder = re.compile('[\s]*\-{2}[\s]*')
commafinder = re.compile('[\s]*,[\s]*')
lollapaloozafinder = re.compile('[_\s]*lollapalooza[_\s]*')
xenakisfinder = re.compile('xenakis')


# If filename has two dashes, band ends before those two dashes
music_df.loc[
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(doubledashfinder, str(x)) != None))
, ['band_abbrev', 'has_pre_date_endmarks']] = [music_df['band_abbrev'][
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(doubledashfinder, str(x)) != None))
].apply(lambda x: x[0:re.search(doubledashfinder, str(x)).start()]), True]

# If filename has comma, band ends before that comma
music_df.loc[
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(commafinder, str(x)) != None))
, ['band_abbrev', 'has_pre_date_endmarks']] = [music_df['band_abbrev'][
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(commafinder, str(x)) != None))
].apply(lambda x: x[0:re.search(commafinder, str(x)).start()]), True]


# Fix "lollapalooza" filenames
music_df.loc[
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(lollapaloozafinder, str(x)) != None))
, ['band_abbrev', 'has_pre_date_endmarks']] = [music_df['band_abbrev'][
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(lollapaloozafinder, str(x)) != None))
].apply(lambda x: x[0:re.search(lollapaloozafinder, x).start()]), True]

music_df.loc[
    (music_df['band_abbrev'].notnull())
    & (music_df['band_abbrev'].apply(lambda x: re.search(xenakisfinder, str(x)) != None))
    , ['band_abbrev', 'has_pre_date_endmarks']] = [music_df['band_abbrev'][
    (music_df['band_abbrev'].notnull()) 
    & (music_df['band_abbrev'].apply(lambda x: re.search(xenakisfinder, str(x)) != None))
].apply(lambda x: x[re.search(xenakisfinder, x).start():re.search(xenakisfinder, x).end()]), True]


# ONE-TIME SPECIAL CASE FIXES
music_df.loc[(music_df['band_abbrev'] == '001a'), ['band_abbrev', 'has_pre_date_endmarks']] = ['', True]
music_df.loc[(music_df['band_abbrev'] == 'acdc uk - dunfermline'), ['band_abbrev', 'has_pre_date_endmarks']] = ['acdc', True]
music_df.loc[(music_df['band_abbrev'] == 'casting crowns royals stadium kansas city missouri'), ['band_abbrev', 'has_pre_date_endmarks']] = ['casting crowns', True]
music_df.loc[(music_df['band_abbrev'] == 'charlie cunningham - graz'), ['band_abbrev', 'has_pre_date_endmarks']] = ['charlie cunningham', True]
music_df.loc[(music_df['band_abbrev'] == 'chess smith trio -poncho seattle'), ['band_abbrev', 'has_pre_date_endmarks']] = ['chess smith trio', True]
music_df.loc[(music_df['band_abbrev'] == 'dizzy mizz lizzy (roskilde'), ['band_abbrev', 'has_pre_date_endmarks']] = ['dizzy mizz lizzy', True]
music_df.loc[(music_df['band_abbrev'] == 'editors glastonbury'), ['band_abbrev', 'has_pre_date_endmarks']] = ['editors', True]
music_df.loc[(music_df['band_abbrev'] == 'eluveitielorley'), ['band_abbrev', 'has_pre_date_endmarks']] = ['eluveitie', True]
music_df.loc[(music_df['band_abbrev'] == 'feldman - rothko chapel (jourdain'), ['band_abbrev', 'has_pre_date_endmarks']] = ['feldman', True]
music_df.loc[(music_df['band_abbrev'] == 'feldman - voices and cello (safir'), ['band_abbrev', 'has_pre_date_endmarks']] = ['feldman', True]
music_df.loc[(music_df['band_abbrev'] == 'front242_metro'), ['band_abbrev', 'has_pre_date_endmarks']] = ['front242', True]
music_df.loc[(music_df['band_abbrev'] == 'glen hansard - rudolstadt'), ['band_abbrev', 'has_pre_date_endmarks']] = ['glen hansard', True]
music_df.loc[(music_df['band_abbrev'] == 'jam- poole'), ['band_abbrev', 'has_pre_date_endmarks']] = ['jam', True]
music_df.loc[(music_df['band_abbrev'] == 'jam_apollo'), ['band_abbrev', 'has_pre_date_endmarks']] = ['jam', True]
music_df.loc[(music_df['band_abbrev'] == 'karma to burn_denver co'), ['band_abbrev', 'has_pre_date_endmarks']] = ['karma to burn', True]
music_df.loc[(music_df['band_abbrev'] == 'keith jarrett solo'), ['band_abbrev', 'has_pre_date_endmarks']] = ['keith jarrett', True]
music_df.loc[(music_df['band_abbrev'] == 'keith jarrett solo osaka'), ['band_abbrev', 'has_pre_date_endmarks']] = ['keith jarrett', True]
music_df.loc[(music_df['band_abbrev'] == 'kiss independence mo events center'), ['band_abbrev', 'has_pre_date_endmarks']] = ['kiss', True]
music_df.loc[(music_df['band_abbrev'] == 'l-ficher-g-baton-adelaide'), ['band_abbrev', 'has_pre_date_endmarks']] = ['l-ficher-g-baton', True]
music_df.loc[(music_df['band_abbrev'] == 'maggie\'s farm ('), ['band_abbrev', 'has_pre_date_endmarks']] = ['maggie\'s farm', True]
music_df.loc[(music_df['band_abbrev'] == 'marillion_denhaag'), ['band_abbrev', 'has_pre_date_endmarks']] = ['marillion', True]
music_df.loc[(music_df['band_abbrev'] == 'memorial for pat and rick'), ['band_abbrev', 'has_pre_date_endmarks']] = ['', True]
music_df.loc[(music_df['band_abbrev'] == 'ministry_riviera'), ['band_abbrev', 'has_pre_date_endmarks']] = ['ministry', True]
music_df.loc[(music_df['band_abbrev'] == 'neil young - orange stage'), ['band_abbrev', 'has_pre_date_endmarks']] = ['neil young', True]
music_df.loc[(music_df['band_abbrev'] == 'peter &amp; the test tube babies - edinburgh'), ['band_abbrev', 'has_pre_date_endmarks']] = ['peter &amp; the test tube babies', True]
music_df.loc[(music_df['band_abbrev'] == 'peter gabriel _ paris'), ['band_abbrev', 'has_pre_date_endmarks']] = ['peter gabriel', True]
music_df.loc[(music_df['band_abbrev'] == 'phish-sleeptrain.cvca'), ['band_abbrev', 'has_pre_date_endmarks']] = ['phish-sleeptrain', True]
music_df.loc[(music_df['band_abbrev'] == 'radio-interview with joe jackson'), ['band_abbrev', 'has_pre_date_endmarks']] = ['joe jackson', True]
music_df.loc[(music_df['band_abbrev'] == 'rob brown quartet jpc clemente ny'), ['band_abbrev', 'has_pre_date_endmarks']] = ['rob brown quartet', True]
music_df.loc[(music_df['band_abbrev'] == 'rrk_village_vanguard'), ['band_abbrev', 'has_pre_date_endmarks']] = ['rrk', True]
music_df.loc[(music_df['band_abbrev'] == 'the clash - rettel festival'), ['band_abbrev', 'has_pre_date_endmarks']] = ['the clash', True]
music_df.loc[(music_df['band_abbrev'] == 'the passenger-iggy pop-isle of wight'), ['band_abbrev', 'has_pre_date_endmarks']] = ['the passenger-iggy pop', True]
music_df.loc[(music_df['band_abbrev'] == 'the ramones-durango'), ['band_abbrev', 'has_pre_date_endmarks']] = ['the ramones', True]
music_df.loc[(music_df['band_abbrev'] == 'the waterboys warsaw'), ['band_abbrev', 'has_pre_date_endmarks']] = ['the waterboys', True]
music_df.loc[(music_df['band_abbrev'] == 'van morrison - montreux'), ['band_abbrev', 'has_pre_date_endmarks']] = ['van morrison', True]
music_df.loc[(music_df['band_abbrev'] == 'vm - birmingham'), ['band_abbrev', 'has_pre_date_endmarks']] = ['vm', True]
music_df.loc[(music_df['band_abbrev'] == 'vm - hamburg'), ['band_abbrev', 'has_pre_date_endmarks']] = ['vm', True]
music_df.loc[(music_df['band_abbrev'] == 'wayne shorter quartet budapest'), ['band_abbrev', 'has_pre_date_endmarks']] = ['wayne shorter quartet', True]
music_df.loc[(music_df['band_abbrev'] == 'where it\'s at - good times - china girl - pocket calculator - 1999 medley ('), ['band_abbrev', 'has_pre_date_endmarks']] = ['where it\'s at - good times - china girl', True]
music_df.loc[(music_df['band_abbrev'] == 'nickcave_metro'), ['band_abbrev', 'has_pre_date_endmarks']] = ['Nick Cave', True]

print('backing up...')
music_df_bk = music_df

music_df.groupby('band_abbrev').size().sort_values(ascending=False)

### Sometimes the band name is AFTER the date. What do we do? Treat as special cases.

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

music_df = music_df.assign(band_abbrev_after_date = np.nan)
music_df.loc[music_df['band_abbrev'].notnull(), 'band_abbrev_after_date'] = False

startmarks = re.compile('^[\.\-\_\ @]+')
starts_with_number = re.compile('^[\d]+[\.\-\_\ \(\)\s]*')
thingy = re.compile('^d[\d]+t[\d]+\.')


bandseps = re.compile('[\s]*[,_\-\.]+[\s]*')


music_df['postdate'][
    (music_df['band_abbrev'] == '')
    & (music_df['postdate'].notnull())
    & (music_df['postdate'].apply(lambda x: re.search(startmarks, str(x)) != None))
].apply(lambda x: x[re.search(startmarks, x).end():len(x)])

music_df.loc[music_df['postdate'] == '  Barry Melton &amp; The Green Ray Sol Party 2016 Track 01', ['band_abbrev', 'band_abbrev_after_date']] = ['Barry Melton &amp; The Green Ray', True]
music_df.loc[music_df['postdate'] == ' (PA)-11 O\'clock tick tock', ['band_abbrev', 'band_abbrev_after_date']] = ['11 O\'clock tick tock', True]
music_df.loc[music_df['postdate'] == ' - 101', ['band_abbrev', 'band_abbrev_after_date']] = ['', True]
music_df.loc[music_df['postdate'] == ' - David Gilmour - Stuttgart T01', ['band_abbrev', 'band_abbrev_after_date']] = ['David Gilmour', True]
music_df.loc[music_df['postdate'] == ' - David Gilmour - Track 01', ['band_abbrev', 'band_abbrev_after_date']] = ['David Gilmour', True]
music_df.loc[music_df['postdate'] == ' - Neil Young - Berlin T01', ['band_abbrev', 'band_abbrev_after_date']] = ['Neil Young', True]
music_df.loc[music_df['postdate'] == ' - Peter, Bjorn &amp; John - Paredes de Coura01', ['band_abbrev', 'band_abbrev_after_date']] = ['Peter, Bjorn, & John', True]
music_df.loc[music_df['postdate'] == ' - This News Is Captured! - Wolverhampton - 101 Firebird Suite', ['band_abbrev', 'band_abbrev_after_date']] = ['This News Is Captured!', True]
music_df.loc[music_df['postdate'] == ' - Van Morrison - Stuttgart T01', ['band_abbrev', 'band_abbrev_after_date']] = ['Van Morrison', True]
music_df.loc[music_df['postdate'] == ' Boris (Pink) The Casbah01', ['band_abbrev', 'band_abbrev_after_date']] = ['Boris', True]
music_df.loc[music_df['postdate'] == ' Clawhammer, The Casbah01', ['band_abbrev', 'band_abbrev_after_date']] = ['Clawhammer', True]
music_df.loc[music_df['postdate'] == ' Cully Jazz, Wayne Shorter Quartet (broadcast)', ['band_abbrev', 'band_abbrev_after_date']] = ['Wayne Shorter Quartet', True]
music_df.loc[music_df['postdate'] == ' Drinks Brighton01', ['band_abbrev', 'band_abbrev_after_date']] = ['Drinks', True]
music_df.loc[music_df['postdate'] == ' Elvin', ['band_abbrev', 'band_abbrev_after_date']] = ['Elvin', True]

music_df.loc[music_df['postdate'] == ' Savages Observatory North Park01', ['band_abbrev', 'band_abbrev_after_date']] = ['Savages', True]
music_df.loc[music_df['postdate'] == ' wsp@the joint_101', ['band_abbrev', 'band_abbrev_after_date']] = ['wsp', True]
music_df.loc[music_df['postdate'] == ' wsp@thejoint_101', ['band_abbrev', 'band_abbrev_after_date']] = ['wsp', True]
music_df.loc[music_df['postdate'] == '-Trembling Bells aud01', ['band_abbrev', 'band_abbrev_after_date']] = ['Trembling Bells', True]
music_df.loc[music_df['postdate'] == '_01_Religion', ['band_abbrev', 'band_abbrev_after_date']] = ['Religion', True]
music_df.loc[music_df['postdate'] == '_Jam_4Acres08', ['band_abbrev', 'band_abbrev_after_date']] = ['Jam', True]
music_df.loc[music_df['postdate'] == '_Jam_Paris_mc01', ['band_abbrev', 'band_abbrev_after_date']] = ['Jam', True]
music_df.loc[music_df['postdate'] == '_Radiohead_Shoreline_25 Everything In Its Right Place', ['band_abbrev', 'band_abbrev_after_date']] = ['Radiohead', True]
music_df.loc[
    (music_df['postdate'] == ' - Century Hall - Nagoya - 101 Rhythm Of Love')
    | (music_df['postdate'] == ' - d3t01 Open All Night')
    | (music_df['postdate'] == ' - 101') | (music_df['postdate'] == ' Ann Arbor MI - 01 Improvisation')
    | (music_df['postdate'] == ' - d3t02 Pay Me My Money Down')
    | (music_df['postdate'] == ' 101') | (music_df['postdate'] == ' Berlin DE - 01 Incan Intro')
    | (music_df['postdate'] == ' Budapest 101') | (music_df['postdate'] == ' Essen Main Part')
    | (music_df['postdate'] == ' Part 1') | (music_df['postdate'] == ' Part II')
    | (music_df['postdate'] == ' Theme-Esther\'s Nose Job') | (music_df['postdate'] == ' d2t04')
    | (music_df['postdate'] == ' d3t02') | (music_df['postdate'] == ' san francisco d2t01')
    | (music_df['postdate'] == ' san francisco d2t06') | (music_df['postdate'] == ' tr20')
    | (music_df['postdate'] == '-0022') | (music_df['postdate'] == '-0028') | (music_df['postdate'] == '-d1-tr01')    
    | (music_df['postdate'] == '_07') | (music_df['postdate'] == '_1851') 
    , ['band_abbrev', 'band_abbrev_after_date']] = [' ', True]

print('backing up...')
music_df_bk = music_df

#for x in music_df['postdate'][
#    (music_df['band_abbrev'] == '')
#    & (music_df['postdate'].notnull())
#    & (music_df['postdate'].apply(lambda x: re.search(startmarks, str(x)) != None))
#    #& (music_df['postdate'].apply(lambda x: str(x)[re.search(startmarks, str(x)).end():len(str(x))] == 'wsp@thejoint_101'))
#].drop_duplicates().sort_values().tolist():
#    print('**'+x+'**')#.apply(lambda x: x[re.search(startmarks, x).end():len(x)]).drop_duplicates().sort_values()

music_df.groupby('band_abbrev_after_date').size()

## Get list of official band abbreviations from etree

In [ ]:
# GET LIST OF OFFICIAL ABBREVIATIONS

print('Retrieving from etree')
pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

column_order = ['name', 'abbrev', 'otherabbrev', 'notes']
abbrev_df = abbrev_df[column_order]
abbrev_df = abbrev_df.fillna('')


# Trim one space from beginning and end of band name
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 

abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])

abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

#abbrev_df['abbrev'].apply(lambda x: '|'+str(x)+'|')

abbrev_df.to_csv('abbreviations.csv')
os.listdir()
#print('ok')
#pprint(abbrev_df.values.tolist())
canonical_abbrevs = abbrev_df['abbrev'][abbrev_df['abbrev'] != ''].drop_duplicates().tolist()
#canonical_abbrevs

# Get variant abbreviations
variant_abbrevs = []
for x in abbrev_df['otherabbrev'].drop_duplicates():
    if (',' in x):
        for y in x.split(','):
            variant_abbrevs.append(y.strip())
    else:
        if (x.strip() != ''):
            variant_abbrevs.append(x.strip())

variant_abbrevs = sorted(variant_abbrevs)

# Fix wilco by moving it from variant to canonical
canonical_abbrevs.append('wilco')
variant_abbrevs.remove('wilco')

# Strip and lower
canonical_abbrevs = [x.lower().strip() for x in canonical_abbrevs]    
variant_abbrevs = [x.lower().strip() for x in variant_abbrevs]    

print('Parsed {0:,.0f} canonical abbreviations and {1:,.0f} variant abbreviations.'.format(len(canonical_abbrevs), len(variant_abbrevs)))
abbrev_df.sample(3)



## Rotate abbrevations into a guide file to apply to bands

In [ ]:
# ROTATE ABBREVIATIONS INTO GUIDE FILE
guide_df = pandas.DataFrame(data=abbrev_df['abbrev'][abbrev_df['abbrev'] != ''], columns=['abbrev'])

guide_df = guide_df.merge(abbrev_df, how='left', on='abbrev')
guide_df = guide_df[['abbrev', 'name']]
guide_df = guide_df.set_index('abbrev')
guide_df['canonical'] = True

variant_abbrev_list = []
for thisrow in abbrev_df['otherabbrev'][abbrev_df['otherabbrev'] != ''].drop_duplicates().tolist():
    for thisabbrev in thisrow.split(','):
        thisabbrev = thisabbrev.strip()
        thisband = abbrev_df['name'][abbrev_df['otherabbrev'].apply(lambda x: thisabbrev in x)].values[0]
        if thisabbrev not in guide_df.index.values:
            guide_df.loc[thisabbrev] = [thisband, False]

guide_df = guide_df.sort_values('canonical').sort_index()

# change index value of kdub
indexaslist = guide_df.index.tolist()
kdubvalue = indexaslist.index('kdub (for k-double-u)')
indexaslist[kdubvalue] = 'kdub'
guide_df.index = indexaslist

# Clean up some weird-ass values
guide_df.loc['beanland', 'name'] = 'Beanland'
guide_df.loc['buho', 'canonical'] = True
guide_df.loc['logic', 'canonical'] = True
guide_df.loc['gtb', 'canonical'] = True
guide_df.loc['ig', 'canonical'] = True
guide_df.loc['jgb', 'name'] = 'Jerry Garcia Band'
guide_df.loc['jgb', 'canonical'] = True
guide_df.loc['jt', 'canonical'] = True
guide_df.loc['ls', 'canonical'] = True
guide_df.loc['los', 'canonical'] = True
guide_df.loc['marlow', 'canonical'] = True
guide_df.loc['moe', 'name'] = 'Moe'
guide_df.loc['pmb', 'canonical'] = True
guide_df.loc['amendola', 'canonical'] = True
guide_df.loc['hip', 'canonical'] = True
guide_df.loc['wilco', 'canonical'] = True
guide_df.loc['spod', 'name'] = 'Serial Pod'

guide_df.index.name = 'abbrev'

guide_df_bk = guide_df
guide_df.sample(3)

# Figure out which band names are canonical, incorrect, etc.

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk
#music_df['band_abbrev']
music_df = music_df.assign(band = music_df.join(guide_df['name'], how='left', on='band_abbrev')['name'])
music_df = music_df.assign(canonical = music_df.join(guide_df['canonical'], how='left', on='band_abbrev')['canonical'])
music_df['canonical'].dropna()
#music_df['band_abbrev'] = music_df['band_abbrev'].astype(str)
#music_df.join(guide_df, how='left')
#music_df.dtypes
#music_df['band_abbrev']
#music_df['band_abbrev'].astype(str)
music_df = music_df.assign(abbrev_type = np.nan)
music_df.loc[music_df['canonical'] == True, 'abbrev_type'] = 'canonical'
music_df.loc[music_df['canonical'] == False, 'abbrev_type'] = 'variant'

# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic', 'jg+ms', 'deadco', 'deadandco', 'dc', 'jgms', 'ph']
incorrect += ['dead and company', 'dead.and.co', 'dead.and.co', 'dac', 'dandc']
incorrect += ['deadco-', 'dead&co', 'dead&amp;co', 'dead&amp;company', 'deadandcompany']
incorrect += ['deadco.irvine.', 'deadfeat', 'gov`t mule', 'gm']
incorrect += ['maggie\'s farm', 'memorial for pat and rick', 'where it\'s at - good times - china girl']
music_df.loc[music_df['band_abbrev'].isin(incorrect), 'abbrev_type'] = 'incorrect'

# No band name
music_df.loc[music_df['band_abbrev'] == '', 'abbrev_type'] = 'blank'

# Band names not in abbreviation list
notfound = ['bd', 'loslobos', 'deanmonkey&amp;thedropouts', 'ttb', 'crb', 'deadandcompany', 'ny', 'gsbg', 'bd']
notfound += ['jefferson starship', 'carlton, larry -', 'the who', 's marriott', 'unknown artist']
notfound += ['tal wilkenfeld', 'ph&amp;tl', 'sb', 'radiohead', 'theclash_', 'primus']
notfound += ['u2', 'theclash', 'humble pie', 'nma', 'raccoon', 'jefferson airplane']
notfound += ['bobdylan', 'ec', 'hawkwind', 'headforthehills', 'mavisstaples', 'pixies']
notfound += ['ramones', 'rhcp', 'tbc', 'the london souls', 'tth']
notfound += ['america', 'bob', 'bush', 'calexico', 'coldplay', 'cornell', 'damo', 'dnc', 'dylan']
notfound += ['gaf', 'gnr', 'halfmoon', 'hazmat.modine', 'heart', 'jerryjoseph', 'mccartney']
notfound += ['mostlyautumn', 'nrps', 'oakhurst', 'ot', 'rem', 'ryanadams', 'sarahpeacock']
notfound += ['spirit', 'suede']
music_df.loc[music_df['band_abbrev'].isin(notfound), 'abbrev_type'] = 'notfound'

# Find additional "notfound" bands
vc = music_df['band_abbrev'][
    (music_df['band_abbrev'] != '')
    & (music_df['abbrev_type'].isnull())
].value_counts()
music_df.loc[music_df['band_abbrev'].isin(vc[vc <= 2].index), 'abbrev_type'] = 'notfound'

print('backing up...')
music_df_bk = music_df

music_df.groupby('abbrev_type').size().sort_values(ascending=False)

## Fix band names

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

music_df.loc[
    (music_df['band'].isnull())
    & (music_df['band_abbrev'].notnull())
    & (music_df['abbrev_type'] == 'blank')    
, 'band'] = '<blank>'

music_df.loc[
    (music_df['band'].isnull())
    & (music_df['band_abbrev'].notnull())   
, 'band'] = music_df['band_abbrev'][(music_df['band'].isnull()) & (music_df['band_abbrev'].notnull())].apply(lambda x: x.capitalize())

# Fixes
music_df.loc[music_df['band'] == 'Loslobos', 'band'] = 'Los Lobos'
music_df.loc[music_df['band'] == 'Widespreadpanic', 'band'] = 'Widespread Panic'
music_df.loc[(music_df['band'] == 'Deanmonkey&amp;thedropouts') | (music_df['band'] == 'Deanmonkey&thedropouts'), 'band'] = 'Dean Monkey & the Dropouts'
music_df.loc[music_df['band'] == 'Ttb', 'band'] = 'Tedeschi trucks band'
music_df.loc[music_df['band'] == 'Crb', 'band'] = 'Chris Robinson Brotherhood'
music_df.loc[music_df['band'] == 'Gsbg', 'band'] = 'Greensky Bluegrass'
music_df.loc[music_df['band'] == 'S marriott', 'band'] = 'Steve Marriott'
music_df.loc[music_df['band'] == 'Theclash', 'band'] = 'The Clash'
music_df.loc[music_df['band'] == 'Sb', 'band'] = 'StringerBessant'
music_df.loc[music_df['band'] == 'Ec', 'band'] = 'EC'
music_df.loc[music_df['band'] == 'Headforthehills', 'band'] = 'Head for the Hills'
music_df.loc[music_df['band'] == 'Mavisstaples', 'band'] = 'Mavis Staples'
music_df.loc[music_df['band'] == 'Nma', 'band'] = 'New Model Army'
music_df.loc[music_df['band'] == 'Gaf', 'band'] = 'GAF'
music_df.loc[music_df['band'] == 'Mostlyautumn', 'band'] = 'Mostly Autumn'
music_df.loc[music_df['band'] == 'Nrps', 'band'] = 'New Riders of the Purple Sage'
music_df.loc[music_df['band'] == 'Ot', 'band'] = 'OT'
music_df.loc[music_df['band'] == 'Rem', 'band'] = 'R.E.M.'
music_df.loc[music_df['band'] == 'Ryanadams', 'band'] = 'Ryan Adams'
music_df.loc[music_df['band'] == 'Sarahpeacock', 'band'] = 'Sarah Peacock'


music_df.loc[
    (music_df['band'] == 'Bd')
    | (music_df['band'] == 'Bob')
    | (music_df['band'] == 'Dylan')
    | (music_df['band'] == 'Bobdylan')
    | (music_df['band'] == 'Bob dylan')
    , 'band'] = 'Bob Dylan'

music_df.loc[
    (music_df['band'] == 'Deadco')
    | (music_df['band'] == 'Deadandcompany')
    | (music_df['band'] == 'Deadandcompany')
    | (music_df['band'] == 'Deadandco')
    | (music_df['band'] == 'Dc')
    | (music_df['band'] == 'Dac')
    | (music_df['band'] == 'Dandc')
    | (music_df['band'] == 'Dnc')
    | (music_df['band'] == 'Dead.and.co')
    | (music_df['band'] == 'Dead and company')    
    , 'band'] = 'Dead & Company'
music_df.loc[
    (music_df['band'] == 'Neil young')
    | (music_df['band'] == 'Neil.young')
    | (music_df['band'] == 'Neilyoung')
    | (music_df['band'] == 'Ny')
    , 'band'] = 'Neil Young'
music_df.loc[
    (music_df['band'] == 'Jg+ms')
    | (music_df['band'] == 'Jgms')
    | (music_df['band'] == 'Jg&ms')    
    | (music_df['band'] == 'Ms_jgb')    
    , 'band'] = 'Jerry Garcia & M.S. <who?>'

music_df.loc[music_df['band'] == 'Aaronkamm', 'band'] = 'Aaron Kamm'
music_df.loc[music_df['band'] == 'Bbking', 'band'] = 'B.B. King'
music_df.loc[music_df['band'] == 'Biffyclyro', 'band'] = 'Biffy Clyro'
music_df.loc[music_df['band'] == 'Big.daddy.wilson', 'band'] = 'Big Daddy Wilson'
music_df.loc[music_df['band'] == 'Bsb', 'band'] = 'Backstreet Boys'
music_df.loc[music_df['band'] == 'Byrds&amp;flying burrito bros', 'band'] = 'The Byrds & The Flying Burrito Brothers'
music_df.loc[music_df['band'] == 'Bromberg', 'band'] = 'Brett Romberg'
music_df.loc[music_df['band'] == 'Davidgilmour', 'band'] = 'David Gilmour'
music_df.loc[music_df['band'] == 'Desertdwellers', 'band'] = 'Desert Dwellers'
music_df.loc[music_df['band'] == 'Deltarae', 'band'] = 'Delta Rae'
music_df.loc[music_df['band'] == 'Dz', 'band'] = 'DZ Deathrays'
music_df.loc[(music_df['band'] == 'Ekb') | (music_df['band'] == 'Erickrasnoband'), 'band'] = 'Eric Krasno Band'
music_df.loc[music_df['band'] == 'Elh', 'band'] = 'ELH'
music_df.loc[music_df['band'] == 'Elisetestone', 'band'] = 'Elise Testone'
music_df.loc[music_df['band'] == 'Emmylou', 'band'] = 'Emmy Lou Harris'
music_df.loc[music_df['band'] == 'Ericburdon', 'band'] = 'Eric Burdon'
music_df.loc[music_df['band'] == 'Erikdeutsch', 'band'] = 'Erik Deutsch'
music_df.loc[music_df['band'] == 'Faunfables', 'band'] = 'Faun Fables'
music_df.loc[music_df['band'] == 'Friendsofthefamily', 'band'] = 'Friends of the Family'

music_df.loc[music_df['band'] == 'Funkfellowship', 'band'] = 'Funk Fellowship'
music_df.loc[music_df['band'] == 'Gtd', 'band'] = 'GTD Band'
music_df.loc[music_df['band'] == 'Hayleyjane', 'band'] = 'Hayley Jane'
music_df.loc[music_df['band'] == 'Jefftobias', 'band'] = 'Jeff Tobias'
music_df.loc[music_df['band'] == 'Joejackson', 'band'] = 'Joe Jackson'
music_df.loc[music_df['band'] == 'Johnnywinter', 'band'] = 'Johnny Winter'
music_df.loc[music_df['band'] == 'Johnprine', 'band'] = 'John Prine'
music_df.loc[music_df['band'] == 'Julierhodes', 'band'] = 'Julie Rhodes'
music_df.loc[music_df['band'] == 'Jwalsh', 'band'] = 'Joe Walsh'
music_df.loc[music_df['band'] == 'Kamasiwashington', 'band'] = 'Kamasi Washington'
music_df.loc[music_df['band'] == 'Larrycampbell_teresawilliams', 'band'] = 'Larry Campbell & Teresa Williams'
music_df.loc[music_df['band'] == 'Lhb', 'band'] = 'LHB'
music_df.loc[music_df['band'] == 'Ll', 'band'] = 'Laura Lisbeth'
music_df.loc[music_df['band'] == 'Londonso', 'band'] = 'London Symphony Orchestra'
music_df.loc[music_df['band'] == 'Lrb', 'band'] = 'Little River Band'
music_df.loc[music_df['band'] == 'Lucindawilliams', 'band'] = 'Lucinda Williams'
music_df.loc[music_df['band'] == 'Marcuskingband', 'band'] = 'Marcus King Band'
music_df.loc[music_df['band'] == 'Marthascanlan', 'band'] = 'Martha Scanlan'
music_df.loc[music_df['band'] == 'Mavis', 'band'] = 'Mavis Staples'
music_df.loc[music_df['band'] == 'Mh', 'band'] = 'M & H Band'
music_df.loc[music_df['band'] == 'Montemontgomery', 'band'] = 'Monte Montgomery'
music_df.loc[music_df['band'] == 'Noelgallagher', 'band'] = 'Noel Gallagher'
music_df.loc[music_df['band'] == 'Samholtband', 'band'] = 'Sam Holt Band'
music_df.loc[music_df['band'] == 'Sherylcrow', 'band'] = 'Sheryl Crow'
music_df.loc[music_df['band'] == 'Tom petty &amp; mudcrutch', 'band'] = 'Tom Petty & Mudcrutch'
music_df.loc[music_df['band'] == 'Vm', 'band'] = 'Van Morrison'
music_df.loc[music_df['band'] == 'Willienelson', 'band'] = 'Willie Nelson'

print('backing up...')
music_df_bk = music_df

#bandcounts_s = music_df.groupby('band').size().sort_values(ascending=False)
#bandcounts[bandcounts == 1].sort_index
music_df.groupby('band').size().sort_values(ascending=False)

## Make sure abbreviation types are correct

In [ ]:
music_df[['date_format','thedate','band_abbrev','band','abbrev_type']].dropna()#.sample(5) # 1507 rows total
music_df.groupby('abbrev_type').size()

##  Preparing the table for output

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

music_df.assign(band_date_no_spacing = np.nan)

music_df.loc[
    (music_df['has_pre_date_endmarks'] == False) 
    & (music_df['band_abbrev_after_date'] == False)
    , 'band_date_no_spacing'] = True

music_df.loc[
    (music_df['has_pre_date_endmarks'] == True)
    | (music_df['band_abbrev_after_date'] == True)
    , 'band_date_no_spacing'] = False


#music_df.groupby('band_date_no_spacing').size()

#music_df[['has_pre_date_endmarks', 'band_abbrev_after_date', 'band_date_no_spacing']][
#    (music_df['has_pre_date_endmarks'].notnull())
#    & (music_df['band_abbrev_after_date'].notnull())
#]

print(music_df.groupby('band_date_no_spacing').size())

column_order = ['id', 'source', 'filename', 'filetype', 'date_format', 'thedate', 'theyear', 'themonth', 'theday']
column_order += ['band_date_no_spacing', 'band_abbrev', 'band', 'abbrev_type']
column_order += ['stem', 'date_string', 'predate', 'postdate', 'has_pre_date_endmarks', 'band_abbrev_after_date', 'canonical']

music_df = music_df[column_order]

music_df

# Write output file. Yay!

In [ ]:
music_df.to_csv('firstpass_data_processed.csv')
print('ok')
x_df = pandas.read_csv('firstpass_data_processed.csv', index_col='rownumber')
x_df.head(3)


In [ ]:
'''
STEPS IN FLOWCHART:
A. Does filename start with the artist name? (False -> 0)
B. Does the date come next? (False -> 0)
C. Does an artist name come next? (False -> 0)

D. Does the artist have an official abbrev? (True -> E, False -> G)

E. Is (abbrev used correctly) OR (date correctly formatted)? (True -> F , False -> 1)
F. Is (abbrev used correctly) AND (date correctly formatted)? (True -> H , False -> 2)

G. Is the date correctly formatted? (True -> H, False -> 1)

H. Does the date immediately follow artist name, no space between? (False -> 3)

J. Do they include the mics parameter? (True -> K, False -> L)
K. Do they use the mics parameter correctly? (True -> L, False -> 4)

L. Is taper used correctly? (False -> 4)
M. Is sbesource used correctly? (False -> 4)
N. Is filetype used correctly? (False -> 4)
P. Does filetype accurately describe the type of file? (True -> 5, False -> 4)
'''
# Researching the .cue file that snuck into the dataset...
#adf = inmusic_df
#adf['Sample File Name'] = adf['Sample File Name'].apply(lambda x: x.lower())
#adf[adf['Sample File Name'].apply(lambda x: '.cue' in x)]
print('ok')